<h1>I have used TF.IDF in order to extract the feature vector from the corpus of text of a particular page in wikipedia</h1>

In [1]:
!pip install wikipedia
import re
import wikipedia

  Created wheel for wikipedia: filename=wikipedia-1.4.0-py3-none-any.whl size=11695 sha256=b0f29e51b3ab63036bcf916324cb4655b269c35b36e033d09d99324b2df7accb
  Stored in directory: /root/.cache/pip/wheels/15/93/6d/5b2c68b8a64c7a7a04947b4ed6d89fb557dcc6bc27d1d7f3ba
Successfully built wikipedia


In [2]:
%%capture
import nltk
nltk.download("all")

In [3]:
from nltk.corpus import stopwords
stop = set(stopwords.words('english'))

In [4]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [5]:
import pandas as pd

In [38]:
data = pd.read_csv("Wiki.csv")
data.head()

,Unnamed: 0,Article_Title,Label,All_Labels,Link
0,0,Absolute Value,1,1,https://en.wikipedia.org/wiki/Absolute_value
1,1,addition theorem,1,1,https://en.wikipedia.org/wiki/Addition_theorem
2,2,Adjoint of a matrix,2,2,https://en.wikipedia.org/wiki/Adjugate_matrix
3,3,Algebra of sets,2,2,https://en.wikipedia.org/wiki/Algebra_of_sets
4,4,Algorithm,2,2,https://en.wikipedia.org/wiki/Algorithm


In [7]:
page_pattern = re.compile(r"([\w\d\s])+")

In [8]:
def listToString(s): 
    # initialize an empty string
    str1 = " " 
    # return string  
    return (str1.join(s))

def page_content(name):
    # Specify the title of the Wikipedia page
    wiki = wikipedia.page(name)

    # Extract the plain text content of the page, excluding images, tables, and other data.
    text = wiki.content
    text = text.replace('==', '')
    text = text.replace('\n', '')[:-12]
    text_words = nltk.word_tokenize(text)
    stop_text = [word for word in text_words if word not in stop]

    valid_token = []
    for word in stop_text:
        if len(word)>2:                
            res = re.match(page_pattern,word)
            if res:
                valid_token.append(word)
    return listToString(valid_token)

In [9]:
def feature_vector(name):
    vectorizer = TfidfVectorizer(max_features=100)
    X = vectorizer.fit_transform([page_content(name)])
    keywords = vectorizer.get_feature_names_out()
    keyword = keywords.tolist()
    return (keyword,X.toarray())

In [10]:
print(feature_vector("Discriminant")[1][0])

[0.05097929 0.03058757 0.06627307 0.03058757 0.0356855  0.13764407
 0.04588136 0.03058757 0.0356855  0.02039171 0.0356855  0.02039171
 0.04588136 0.08666479 0.03058757 0.15803579 0.0356855  0.05607722
 0.29567986 0.56077215 0.0356855  0.0356855  0.02039171 0.04588136
 0.0356855  0.04078343 0.02548964 0.03058757 0.02039171 0.04588136
 0.08156686 0.0356855  0.02039171 0.02548964 0.03058757 0.08156686
 0.02039171 0.02039171 0.02039171 0.02039171 0.04588136 0.03058757
 0.02039171 0.11215443 0.02039171 0.04078343 0.02548964 0.02039171
 0.03058757 0.02548964 0.04078343 0.02548964 0.06627307 0.09686064
 0.05607722 0.06117514 0.03058757 0.02548964 0.071371   0.06627307
 0.07646893 0.02039171 0.02039171 0.04588136 0.06117514 0.41803015
 0.05097929 0.06627307 0.04588136 0.03058757 0.11725236 0.03058757
 0.02548964 0.0356855  0.18352543 0.05097929 0.05097929 0.02039171
 0.02039171 0.06117514 0.24979851 0.02039171 0.02548964 0.03058757
 0.02039171 0.04078343 0.02039171 0.08156686 0.08666479 0.0305

In [39]:
data.reset_index()
data.columns

Index(['Unnamed: 0', 'Article_Title', 'Label', 'All_Labels', 'Link'], dtype='object')

In [40]:
data = data[['Article_Title', 'Label']]

In [41]:
data.head()

,Article_Title,Label
0,Absolute Value,1
1,addition theorem,1
2,Adjoint of a matrix,2
3,Algebra of sets,2
4,Algorithm,2


Getting the feature vectors of all of them


In [14]:
from numpy import nan

In [15]:
fv = []
for index, row in data.iterrows():
    try:
        fv.append(feature_vector(row["Article_Title"])[1][0])
    except:
        fv.append(nan)

/usr/local/lib/python3.7/dist-packages/wikipedia/wikipedia.py:389: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 389 of the file /usr/local/lib/python3.7/dist-packages/wikipedia/wikipedia.py. To get rid of this warning, pass the additional argument 'features="lxml"' to the BeautifulSoup constructor.

  lis = BeautifulSoup(html).find_all('li')
/usr/local/lib/python3.7/dist-packages/wikipedia/wikipedia.py:389: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that c

In [42]:
data["fetaure_vector"] = fv

In [43]:
data.dropna(inplace=True)

In [44]:
data.reset_index()

,index,Article_Title,Label,fetaure_vector
0,0,Absolute Value,1,"[0.021801611683155447, 0.5232386803957307, 0.0..."
1,1,addition theorem,1,"[0.28968272975116055, 0.3379631847096873, 0.04..."
2,2,Adjoint of a matrix,2,"[0.807184978062317, 0.03011884246501183, 0.030..."
3,3,Algebra of sets,2,"[0.06814662756363819, 0.034073313781819096, 0...."
4,4,Algorithm,2,"[0.050770018251899945, 0.5194163405771303, 0.4..."
...,...,...,...,...
408,457,Vector Space,1,"[0.029472616006957542, 0.12860777893945108, 0...."
409,458,Vieta Formula,1,"[0.0552532101512818, 0.1105064203025636, 0.036..."
410,459,Wald-Wolfowitz Run Test,1,"[0.041030496993110906, 0.041030496993110906, 0..."
411,461,Weighted Arithmetic Mean,1,"[0.02496491773433435, 0.0499298354686687, 0.06..."


In [45]:
index_to_be_removed = []
for index, rows in data.iterrows():
    if (type(rows["fetaure_vector"]) == nan) or (len(rows["fetaure_vector"]) < 100):
        index_to_be_removed.append(index)
data = data.drop(index_to_be_removed)

In [46]:
import numpy as np

In [47]:
fv_list = []
for index, row in data.iterrows():
    fv_list.append(np.array(row["fetaure_vector"]))

In [48]:
fv_list

[array([0.02180161, 0.52323868, 0.02180161, 0.03633602, 0.02906882,
        0.03633602, 0.02180161, 0.02180161, 0.02906882, 0.03633602,
        0.02180161, 0.02180161, 0.02180161, 0.05813763, 0.02906882,
        0.02906882, 0.02180161, 0.23255052, 0.03633602, 0.02180161,
        0.07267204, 0.07267204, 0.02906882, 0.02906882, 0.04360322,
        0.03633602, 0.02906882, 0.12354247, 0.02180161, 0.02180161,
        0.02906882, 0.02180161, 0.07267204, 0.02180161, 0.02906882,
        0.04360322, 0.02180161, 0.02906882, 0.04360322, 0.03633602,
        0.03633602, 0.04360322, 0.1889473 , 0.03633602, 0.02180161,
        0.06540484, 0.02906882, 0.02180161, 0.04360322, 0.02180161,
        0.03633602, 0.02906882, 0.02906882, 0.02906882, 0.02906882,
        0.02180161, 0.03633602, 0.02180161, 0.02180161, 0.04360322,
        0.03633602, 0.04360322, 0.12354247, 0.03633602, 0.04360322,
        0.1816801 , 0.1816801 , 0.02180161, 0.03633602, 0.02906882,
        0.06540484, 0.02180161, 0.02180161, 0.06

In [49]:
import numpy as np

In [50]:
x_train = np.array(fv_list)

In [51]:
x_train.shape

(394, 100)

In [52]:
x_train

array([[0.02180161, 0.52323868, 0.02180161, ..., 0.04360322, 0.02180161,
        0.05813763],
       [0.28968273, 0.33796318, 0.04828045, ..., 0.09656091, 0.09656091,
        0.04828045],
       [0.80718498, 0.03011884, 0.03011884, ..., 0.03011884, 0.04216638,
        0.03614261],
       ...,
       [0.0410305 , 0.0410305 , 0.0410305 , ..., 0.0410305 , 0.0410305 ,
        0.16412199],
       [0.02496492, 0.04992984, 0.06865352, ..., 0.04992984, 0.39943868,
        0.35575008],
       [0.08033813, 0.04016907, 0.0602536 , ..., 0.20084533, 0.04016907,
        0.04016907]])

In [53]:
y_list = data["Label"].reset_index()

<h1>Defining the neural network. There are a total of 400 links in the training data. I have divided that into 320 and 80 part as training data and validataion data</h1>

In [54]:
y_list = np.array(data["Label"])
y_list

array([1, 1, 2, 2, 2, 2, 1, 2, 3, 1, 1, 2, 2, 2, 3, 1, 1, 1, 3, 2, 1, 0,
       1, 2, 1, 2, 2, 3, 2, 1, 3, 1, 1, 0, 1, 2, 2, 0, 2, 2, 0, 0, 1, 1,
       2, 2, 0, 0, 2, 1, 2, 2, 1, 1, 2, 2, 1, 1, 1, 2, 1, 2, 2, 1, 2, 1,
       2, 1, 3, 2, 1, 1, 2, 2, 3, 0, 2, 1, 2, 0, 2, 0, 1, 1, 2, 2, 2, 2,
       2, 0, 2, 0, 1, 0, 3, 2, 1, 2, 0, 2, 0, 1, 1, 3, 0, 1, 2, 3, 2, 0,
       2, 1, 1, 1, 1, 1, 3, 2, 0, 1, 1, 2, 3, 0, 1, 2, 1, 3, 1, 1, 1, 2,
       2, 1, 3, 1, 2, 0, 0, 1, 0, 0, 3, 0, 3, 1, 1, 1, 3, 2, 1, 1, 1, 0,
       2, 1, 3, 1, 1, 1, 1, 2, 2, 0, 2, 1, 3, 1, 2, 2, 3, 0, 1, 1, 2, 2,
       2, 3, 1, 3, 2, 2, 0, 1, 2, 1, 3, 3, 1, 2, 2, 2, 3, 2, 2, 2, 2, 2,
       2, 2, 0, 3, 2, 3, 0, 0, 1, 1, 1, 1, 1, 2, 0, 2, 0, 2, 2, 3, 1, 2,
       1, 2, 1, 1, 2, 2, 0, 1, 1, 1, 1, 0, 3, 2, 2, 0, 0, 3, 1, 2, 2, 0,
       1, 1, 0, 1, 2, 0, 0, 1, 2, 2, 1, 2, 3, 3, 1, 2, 1, 2, 1, 2, 0, 1,
       3, 1, 1, 0, 2, 1, 0, 1, 2, 2, 2, 2, 1, 2, 1, 1, 1, 3, 2, 1, 2, 2,
       1, 2, 3, 2, 1, 0, 0, 0, 1, 2, 1, 0, 1, 1, 1,

In [55]:
y_list[y_list == ' '] = 0

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  """Entry point for launching an IPython kernel.


In [56]:
y_train = []
for i in range(len(y_list)):
    arr = np.zeros(4)
    arr[int(y_list[i])] = 1
    y_train.append(arr)

In [57]:
y_train = np.array(y_train)

In [58]:
y_train.shape

(394, 4)

In [59]:
from sklearn.utils import shuffle
x_train, y_train = shuffle(x_train, y_train)

In [60]:
x_train_1 = x_train[:320, :]
x_valid_1 = x_train[320:, :]
x_train_1.shape, x_valid_1.shape

((320, 100), (74, 100))

In [61]:
y_train_1 = y_train[:320, :] 
y_valid_1 = y_train[320:, :]
y_train_1.shape, y_valid_1.shape

((320, 4), (74, 4))

<h1>This is the structure of the neural network that i have</h1>

In [62]:
import tensorflow.keras as keras
num_categories = 4

In [63]:
from tensorflow.keras.models import Sequential

model = Sequential()
from tensorflow.keras.layers import Dense
model.add(Dense(units=100, activation='relu', input_shape=(100,)))
model.add(Dense(units = 30, activation='relu'))
model.add(Dense(units = 4, activation='softmax'))
model.summary()
model.compile(loss='categorical_crossentropy', metrics=['accuracy'])

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 100)               10100     
                                                                 
 dense_1 (Dense)             (None, 30)                3030      
                                                                 
 dense_2 (Dense)             (None, 4)                 124       
                                                                 
Total params: 13,254
Trainable params: 13,254
Non-trainable params: 0
_________________________________________________________________


In [64]:
history = model.fit(
    x_train_1, y_train_1, epochs=10, verbose=1, validation_data=(x_valid_1, y_valid_1)
)

Epoch 1/10
10/10 [==============================] - 1s 25ms/step - loss: 1.3261 - accuracy: 0.3594 - val_loss: 1.2753 - val_accuracy: 0.4324
Epoch 2/10
10/10 [==============================] - 0s 6ms/step - loss: 1.2650 - accuracy: 0.4125 - val_loss: 1.2656 - val_accuracy: 0.4324
Epoch 3/10
10/10 [==============================] - 0s 6ms/step - loss: 1.2445 - accuracy: 0.4531 - val_loss: 1.2604 - val_accuracy: 0.4459
Epoch 4/10
10/10 [==============================] - 0s 4ms/step - loss: 1.2369 - accuracy: 0.4594 - val_loss: 1.2628 - val_accuracy: 0.4189
Epoch 5/10
10/10 [==============================] - 0s 6ms/step - loss: 1.2217 - accuracy: 0.5000 - val_loss: 1.2628 - val_accuracy: 0.4595
Epoch 6/10
10/10 [==============================] - 0s 4ms/step - loss: 1.2065 - accuracy: 0.5000 - val_loss: 1.2667 - val_accuracy: 0.3649
Epoch 7/10
10/10 [==============================] - 0s 7ms/step - loss: 1.1916 - accuracy: 0.5063 - val_loss: 1.2667 - val_accuracy: 0.3649
Epoch 8/10
10/10 [=

In [65]:
!rm -rf drive/MyDrive/saved_model
!mkdir -p saved_model
model.save('saved_model/my_model')

INFO:tensorflow:Assets written to: saved_model/my_model/assets


In [ ]:
!mv my_model drive/MyDrive

In [67]:
fv = feature_vector("Formal proof")[1]
np.argmax(model.predict(fv))

2